In [1]:
from pyspark.sql import SparkSession

MAX_MEMORY = '8g'

spark = SparkSession.builder.appName('taxi-fare-prediction_2nd') \
    .config('spark.driver.memory', MAX_MEMORY) \
    .config('spark.executor.memory', MAX_MEMORY) \
    .getOrCreate()

In [2]:
import os

cwd = os.getcwd()
trip_data_path = os.path.join(cwd, 'learning_spark_data', 'trips', '*.csv')
file_path = f"file:///{trip_data_path.replace(os.sep,'/')}"
trip_df = spark.read.csv(file_path, inferSchema = True, header = True)
trip_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [3]:
trip_df.createOrReplaceTempView('trips')

In [4]:
query = """
SELECT 
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""

In [5]:
data_df = spark.sql(query)
data_df.show(5)

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|              0|               138|                265|         16.5|          0|     Monday|       70.07|
|              1|                68|                264|         1.13|          0|     Monday|       11.16|
|              1|               239|                262|         2.68|          0|     Monday|       18.59|
|              1|               186|                 91|         12.4|          0|     Monday|        43.8|
|              2|               132|                265|          9.7|          0|     Monday|        32.3|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
only showing top 5 rows



# 모델 만들기

In [6]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed = 1)

# 파이프라인 생성
- 전처리 과정을 각 스테이지로 정의해서 쌓는다
- 범주형] StringIndexer + onehotencoding: 'pickup_location_id', 'dropoff_location_id', 'day_of_week'
- 수치형] StandardScaler: 'passenger_count', 'trip_distance', 'pickup_time'

In [7]:
stages = []

In [8]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

cat_features = ['pickup_location_id', 'dropoff_location_id', 'day_of_week']
for cat in cat_features:
    cat_index = StringIndexer(inputCol=cat, outputCol=cat+'_idx').setHandleInvalid('keep')
    onehot_encode = OneHotEncoder(inputCols=[cat_index.getOutputCol()], # _idx col
                                  outputCols=[cat+'_onehot'] # postfix
                                 )
    stages += [cat_index, onehot_encode] #collist
stages

[StringIndexer_458f3c1d9d2a,
 OneHotEncoder_eee454887032,
 StringIndexer_0eb60afe3124,
 OneHotEncoder_65e608f7ce88,
 StringIndexer_33aa930c9790,
 OneHotEncoder_d532bd96969d]

In [9]:
from pyspark.ml.feature import StandardScaler, VectorAssembler

num_features = ['passenger_count', 'trip_distance', 'pickup_time']

for num in num_features:
    num_assembler = VectorAssembler(inputCols=[num], outputCol=num+'_vector')
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol=num+'_scaled')
    stages += [num_assembler, num_scaler]

stages

[StringIndexer_458f3c1d9d2a,
 OneHotEncoder_eee454887032,
 StringIndexer_0eb60afe3124,
 OneHotEncoder_65e608f7ce88,
 StringIndexer_33aa930c9790,
 OneHotEncoder_d532bd96969d,
 VectorAssembler_89b6eb297476,
 StandardScaler_0907621cd877,
 VectorAssembler_3cb1fdf0d805,
 StandardScaler_fcda2a638f59,
 VectorAssembler_5cc1df27def9,
 StandardScaler_09461a408363]

In [11]:
assembler_input = [cat+'_onehot' for cat in cat_features] + [num+'_scaled' for num in num_features]
assembler_input

['pickup_location_id_onehot',
 'dropoff_location_id_onehot',
 'day_of_week_onehot',
 'passenger_count_scaled',
 'trip_distance_scaled',
 'pickup_time_scaled']

In [12]:
assembler = VectorAssembler(inputCols=assembler_input, outputCol='feature_vector')
stages += [assembler]
stages

[StringIndexer_458f3c1d9d2a,
 OneHotEncoder_eee454887032,
 StringIndexer_0eb60afe3124,
 OneHotEncoder_65e608f7ce88,
 StringIndexer_33aa930c9790,
 OneHotEncoder_d532bd96969d,
 VectorAssembler_89b6eb297476,
 StandardScaler_0907621cd877,
 VectorAssembler_3cb1fdf0d805,
 StandardScaler_fcda2a638f59,
 VectorAssembler_5cc1df27def9,
 StandardScaler_09461a408363,
 VectorAssembler_221f4fc01e6c]

In [13]:
from pyspark.ml import Pipeline

pipline = Pipeline(stages=stages)
fitted_transform = pipline.fit(train_df)
vtrain_df = fitted_transform.transform(train_df)
vtrain_df.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vector: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vector: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vector: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (nul

- 선형회귀 모델 - 값들이 수치형
- data_df = feature(6) + label(1)
- cat_feature = 3 => StringIndex + OnehotEncoding
- num_feature = 3 => VectorAssem + standardScaler


In [2]:
# spark.stop()